In [97]:
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, cross_val_score

In [98]:
from fact.io import read_h5py
exec(open('/home/msackel/Desktop/gammaClassification/programm/theta_cut/theta_cut.py').read())
exec(open('/home/msackel/Desktop/gammaClassification/programm/model_significance/model_significance.py').read())

with open('/home/msackel/Desktop/gammaClassification/config/feature.yaml') as f:
        feature = yaml.load(f)

eval_data = read_h5py(
                '/home/msackel/Desktop/gammaClassification/data/raw_data/crab_precuts.hdf5',
                key='events',
                columns=list(feature) + [
                        'theta_deg',
                        'theta_deg_off_1',
                        'theta_deg_off_2',
                        'theta_deg_off_3',
                        'theta_deg_off_4',
                        'theta_deg_off_5',
                        ]
                )

print('---Theta**2 = 0.5')
train_data = theta_cut('/home/msackel/Desktop/gammaClassification/data/raw_data/gamma_precuts.hdf5',
                    '/home/msackel/Desktop/gammaClassification/data/raw_data/crab_precuts.hdf5', 0.5)

---Theta**2 = 0.5


In [99]:
data = pd.concat([train_data.query('label == 1')[:100000], train_data.query('label == 0')[:100000]])

# Best depth 5 score

In [103]:
parameters = {
    'max_depth':5,
    'min_child_weight':0.8,
    'gamma': 0.1,
    'subsample':0.5,
    'colsample_bytree':.9,
    'learning_rate':0.1,
    'nthread':25,
    'objective':'binary:logistic', 
    'scoring':'roc_auc'
}
rnd = XGBClassifier(**parameters)
cross_val_score(rnd, data.drop('label', axis=1), data.label, scoring='roc_auc')

array([ 0.88092333,  0.87990479,  0.88650168])